In [1]:
from numba import cuda, jit, int64
import numpy as np
import timeit, random, time

## Adding a value to each element in an array

In [2]:
def normal_function(arr):
    for i in range(len(arr)):
        arr[i] += 10
        
def vectorized_function(arr):
    arr += 10

@cuda.jit
def numba_kernel(arr):
    index = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
    stride = cuda.blockDim.x * cuda.gridDim.x
    for i in range(index, len(arr), stride):
        arr[i] += 10

In [3]:
N = 1000000
a = np.zeros(N)
blockSize = 256
numBlocks = (N + blockSize - 1) // blockSize

In [4]:
%timeit numba_kernel[numBlocks, blockSize](a)

26.8 ms ± 989 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit normal_function(a)

2.24 s ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit vectorized_function(a)

3.04 ms ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Dot Product

In [38]:
def normal_dot_function(arr1, arr2):
    ret = 0
    for i in range(len(arr1)):
        ret += arr1[i] * arr2[i]
        # print(arr1[i] * arr2[i], end=" ")
    return ret

def numpy_dot_function(arr1, arr2):
    return np.dot(arr1, arr2)

@cuda.jit
def naive_gpu_dot_function(arr1, arr2, s):
    index = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    stride = cuda.blockDim.x * cuda.gridDim.x
    for i in range(index, len(arr1), stride):
        s[i] = arr1[i] * arr2[i]

def naive_gpu_dot_function_runner(arr1, arr2):
    blockSize = 512
    s = np.array([0 for _ in range(len(arr1))])
    numBlocks = (len(arr1) + blockSize - 1) // blockSize
    naive_gpu_dot_function[numBlocks, blockSize](arr1, arr2, s)
    return sum(s)
    

@cuda.jit
def gpu_dot_function(arr1, arr2, s, blockSize):
    cache = cuda.shared.array(shape = 512, dtype = int64) # Change shape depending on blockSize
    i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    tid = cuda.threadIdx.x
    cache[tid] = 0
    N = len(arr1)
    while (i < N):
        cache[tid] += arr1[i] * arr2[i]
        i += cuda.gridDim.x * cuda.blockDim.x
        
    cuda.syncthreads()
    
    i = blockSize // 2
    while (i > 0):
        if (tid < i) and (tid + i < N):
            cache[tid] += cache[tid + i]
        cuda.syncthreads()
        i = i // 2
    
    if (tid == 0):
        cuda.atomic.add(s, 0, cache[0])
    
def gpu_dot_function_runner(arr1, arr2):
    s = np.array([0])
    blockSize = 512
    numBlocks = (len(arr1) + blockSize - 1) // blockSize
    gpu_dot_function[numBlocks, blockSize](arr1, arr2, s, blockSize)
    return s[0]


In [39]:
N = 1000000
x = np.array([random.randint(1, 5) for _ in range(N)])
y = np.array([random.randint(1, 5) for _ in range(N)])

In [ ]:
%timeit normal_dot_function(x, y)

1.83 s ± 37.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit numpy_dot_function(x, y)

3.36 ms ± 80.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit gpu_dot_function_runner(x, y)

45.7 ms ± 333 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit naive_gpu_dot_function_runner(x, y)

1.16 s ± 65.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Matrix Multiplication

In [2]:
def numpy_matmul(arr1, arr2):
    return np.matmul(arr1, arr2)

def normal_matmul(arr1, arr2, res):
    for i in range(len(arr1)):
        for j in range(len(arr2[0])):
            for k in range(len(arr2)):
                res[i][j] += arr1[i][k] * arr2[k][j]
    return res

@cuda.jit
def naive_gpu_matmul(arr1, arr2, res):
    i, j = cuda.grid(2)
    if i < res.shape[0] and j < res.shape[1]:
        tmp = 0.
        for k in range(arr1.shape[1]):
            tmp += arr1[i, k] * arr2[k, j]
        res[i, j] = tmp
        
def naive_gpu_matmul_runner(arr1, arr2, res):
    naive_gpu_matmul[(res.shape[0], res.shape[1]), 1](arr1, arr2, res)
    return res

In [3]:
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=int64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=int64)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp
    
def fast_gpu_matmul_runner(arr1, arr2, res):
    fast_matmul[1, (TPB, TPB)](arr1, arr2, res)
    return res

In [9]:
N, M = 100, 100
arr1 = np.array([[random.randint(1, 5) for _ in range(N)] for _ in range(M)])
arr2 = np.array([[random.randint(1, 5) for _ in range(M)] for _ in range(N)])
res = np.array([[0 for _ in range(len(arr1))] for _ in range(len(arr2[0]))])

In [10]:
%timeit numpy_matmul(arr1, arr2)

2.88 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
res = np.array([[0 for _ in range(len(arr1))] for _ in range(len(arr2[0]))])
%timeit normal_matmul(arr1, arr2, res)

5.05 s ± 65.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
res = np.array([[0 for _ in range(len(arr1))] for _ in range(len(arr2[0]))])
%timeit naive_gpu_matmul_runner(arr1, arr2, res)

38.2 ms ± 3.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
res = np.array([[0 for _ in range(len(arr1))] for _ in range(len(arr2[0]))])
%timeit fast_gpu_matmul_runner(arr1, arr2, res)

7.03 ms ± 42.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Convolutions

In [ ]:
sharpen_matrix = np.array([
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]
])

blur_matrix = np.array([
    [0.0625, 0.125, 0.0625],
    [0.125, 0.25, 0.125],
    [0.0625, 0.125, 0.0625]
])

outline_matrix = np.array([
    [-1, -1, -1],
    [-1, 8, -1],
    [-1, -1, -1]
])

In [ ]:
def normal_convolve(img, kernel): # Both inputs are np.arrays
    # Assuming a rectangular image
    tgt_size = calculate_target_size(
        img_size=img.shape[0],
        kernel_size=kernel.shape[0]
    )
    # To simplify things
    k = kernel.shape[0]
    
    # 2D array of zeros
    convolved_img = np.zeros(shape=(tgt_size, tgt_size))
    
    # Iterate over the rows
    for i in range(tgt_size):
        # Iterate over the columns
        for j in range(tgt_size):
            # img[i, j] = individual pixel value
            # Get the current matrix
            mat = img[i:i+k, j:j+k]
            
            # Apply the convolution - element-wise multiplication and summation of the result
            # Store the result to i-th row and j-th column of our convolved_img array
            convolved_img[i, j] = np.sum(np.multiply(mat, kernel))
            
    return convolved_img